# Saving a DataFrame in Parquet format

- When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The `Parquet` format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

- In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

- The `spark` object and the `df1` and `df2` DataFrames have been setup for you.

## Instructions

- View the row count of `df1` and `df2`.
- Combine `df1` and `df2` in a new DataFrame named `df3` with the `union` method.
- Save `df3` to a parquet file named `AA_DFW_ALL.parquet`.
- Read the `AA_DFW_ALL.parquet` file and show the count.

In [9]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [10]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [13]:
df1 = spark.read.format('csv').load('file:///home/talentum/shared/3_UnderstandingParquet/Dataset/AA_DFW_2017_Departures_Short.csv.gz') # AA_DFW_2017_Departures_Short.csv
df2 = spark.read.format('csv').load('file:///home/talentum/shared/3_UnderstandingParquet/Dataset/AA_DFW_2018_Departures_Short.csv.gz') # AA_DFW_2018_Departures_Short.csv
#/home/talentum/shared/3_UnderstandingParquet/Dataset
df1.schema
df2.schema

# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)
print(df3.printSchema())
df3 = df3.withColumnRenamed('_c3', 'flight_duration')
print(df3.printSchema())

# Save the df3 DataFrame in Parquet format
df3.write.parquet('file:///home/talentum/test-jupyter/P3/M1/SM3/AA_DFW_ALL.parquet', mode='overwrite') #/home/talentum/test-jupyter/P3/M1

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('file:///home/talentum/test-jupyter/P3/M1/SM3/AA_DFW_ALL.parquet').count())
print(spark.read.parquet('file:///home/talentum/test-jupyter/P3/M1/SM3/AA_DFW_ALL.parquet').printSchema())

df1 Count: 139359
df2 Count: 119911
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

None
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- flight_duration: string (nullable = true)

None
259270
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- flight_duration: string (nullable = true)

None
